In [162]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd

In [163]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
sns.set()

# CUSTOM LAYERS

In [164]:
class InputLayer(tf.keras.layers.Layer):
  def __init__(self, name):
    super(InputLayer, self).__init__(name=name)
    self.a = tf.Variable(initial_value=tf.zeros(shape=[2,14]), trainable=False, name= self.name + "_activation")
    
  def call(self, inputs):
    a1a1 = tf.reshape(inputs[0],shape=(1,1))   #1
    a1a2 = tf.reshape(inputs[1],shape=(1,1))   #2 
    a1a2a2 = tf.reshape(inputs[2],shape=(1,1)) #3
    a1a2a1 = tf.reshape(inputs[3],shape=(1,1)) #4
    a2a2 = tf.reshape(inputs[4],shape=(1,1))   #5
    a2a1 = tf.reshape(inputs[5],shape=(1,1))   #6
    a2a1a1 = tf.reshape(inputs[6],shape=(1,1)) #7
    a2a1a2 = tf.reshape(inputs[7],shape=(1,1)) #8

    to_hidden = []
    to_hidden.append((tf.concat([a1a2a2, a2a2], axis = 1)))   #["TrRi"] [3,5]
    to_hidden.append((tf.concat([a1a2a1, a2a1], axis = 1)))   #["RiTr"] [4,6] 
    to_hidden.append((tf.concat([a1a1, a2a1], axis = 1)))     #["Plu"]  [1,6] 
    to_hidden.append((tf.concat([a1a2, a1a2a2], axis = 1)))   #["TTra"] [2,3] 
    to_hidden.append((tf.concat([a2a1, a2a1a1], axis = 1)))   #["TTrb"] [6,7] 
    to_hidden.append((tf.concat([a1a2, a2a1], axis = 1)))     #["Tr"]   [2,6]
    to_hidden.append((tf.concat([a1a2a1, a2a1a2], axis = 1))) #["MeTr"] [4,8] 
    to_hidden.append((tf.concat([a1a2a1, a1a2], axis = 1)))   #["MoTr"] [4,2] 
    to_hidden.append((tf.concat([a1a1, a1a2a1], axis = 1)))   #["RiRi"] [1,4] 
    to_hidden.append((tf.concat([a1a1, a1a2], axis = 1)))     #["MoFo"] [1,2] 
    to_hidden.append((tf.concat([a1a2a2, a1a2a1], axis = 1))) #["MMof"] [3,4] 
    to_hidden.append((tf.concat([a1a1, a1a2a2], axis = 1)))   #["MoRi"] [1,3] 
    to_hidden.append((tf.concat([a1a2a2, a2a1a1], axis = 1))) #["MeRi"] [3,7] 
    to_hidden.append((tf.concat([a1a1, a2a2], axis = 1)))     #["Ri"]   [1,5] 

    output = tf.concat(to_hidden,axis = 0)
    output = tf.transpose(output)
    self.a.assign(output)
    return self.a.value()

In [165]:
class HiddenLayer(tf.keras.layers.Layer):
  def __init__(self, name, weights_init_val, bias_init_val):
    super(HiddenLayer, self).__init__(name = name)
    self.units = 14
    self.weights_init_val = weights_init_val
    self.bias_init_val = bias_init_val

  def build(self, input_shape):
    shape = [int(input_shape[0]), self.units]
    self.w = self.add_weight(initializer = tf.initializers.Constant(self.weights_init_val) ,
                              shape= shape,
                              name = self.name + "_weights")
    self.b = self.add_weight(initializer = tf.initializers.Constant(self.bias_init_val) ,
                              shape= [1, self.units],
                              name = self.name + "_biases")

    self.deltaA = tf.Variable(initial_value=tf.zeros(shape = [1, self.units]),trainable=False, name= self.name + "_adjustements")
    self.a = tf.Variable(initial_value=tf.zeros(shape = [1, self.units]),trainable=False, name= self.name + "_activation")

  def call(self, inputs, temperature):
     # i pesi dei neuroni sono considerati in colonna
    net = tf.math.reduce_sum(inputs * self.w, axis=0) 
    self.a.assign( 1/( 1 + tf.exp( -( (net + self.b)/temperature)) ) )
    return self.a.value()


In [166]:
class OutputLayer(tf.keras.layers.Layer):
  def __init__(self, name, weights_init_val, bias_init_val):
    super(OutputLayer, self).__init__(name = name)
    self.units = 14
    self.weights_init_val = weights_init_val
    self.bias_init_val = bias_init_val

  def build(self, input_shape):
    shape = [int(input_shape[-1]), self.units]
    self.w = self.add_weight(initializer = tf.initializers.Constant(self.weights_init_val) ,
                              shape= shape,
                              name = self.name + "_weights")
    self.b = self.add_weight(initializer = tf.initializers.Constant(self.bias_init_val) ,
                              shape= [1, self.units],
                              name = self.name + "_biases")

    self.deltaA = tf.Variable(initial_value=tf.zeros(shape = [1, self.units]),trainable=False, name= self.name + "_adjustements")
    self.a = tf.Variable(initial_value=tf.zeros(shape = [1, self.units]),trainable=False, name= self.name + "_activation")

  def call(self, inputs, temperature):
     # i pesi dei neuroni sono considerati in colonna
     
    net = tf.matmul(inputs,self.w) 
    self.a.assign( 1/( 1 + tf.exp( -( (net + self.b)/temperature)) ) )
    return self.a.value()

---

# CUSTOM MODEL

In [167]:
class MQSelfReflexiveNetwork(tf.keras.Model):
  def __init__(self, learning_rate, initial_temperature, bias_value):
    super(MQSelfReflexiveNetwork, self).__init__(name='MQSelfReflexiveNetwork')

    self.input_layer = InputLayer(name = "InputLayer")
    self.h = HiddenLayer(name = "HiddenLayer",weights_init_val=0.5, bias_init_val=bias_value)
    self.o = OutputLayer(name = "OutputLayer",weights_init_val=0.59, bias_init_val=bias_value)
    self.temperature = tf.Variable(initial_value = initial_temperature,trainable=False, name = "Temperature")
    self.error = tf.Variable(initial_value=0.0,trainable=False, name = "Error")
    self.alpha = learning_rate

  def call(self, input):
    ### FOWARD PASS
    res_input = self.input_layer(input)
    res_hidden = self.h(res_input, self.temperature)
    res_output = self.o(res_hidden, self.temperature)
    return tf.math.reduce_sum(tf.math.square(res_hidden - res_output))
  
  def train_step(self, input):
    ### CALL FOWARD PASS
    self.error.assign(self(input, training = True))

    ### BACK PROPAGATION
    
    # Compute temperature
    #self.temperature.assign(1 - (1 / (1 + self.error) ))

    # Update outputs weights
    delta_Ao = self.o.a * (self.h.a - self.o.a) * (1 - self.o.a) + (self.o.deltaA * ((self.h.a - self.o.a)) )
    delta_Wo = self.alpha * tf.matmul(tf.transpose(self.h.a),delta_Ao)
    self.o.deltaA.assign(delta_Ao)

    ## Update hidden weights
    delta_Ah = self.h.a * (1 - self.h.a) * tf.matmul(self.o.deltaA, self.o.w)
    delta_Wh = self.alpha * delta_Ah * self.input_layer.a
    self.h.deltaA.assign(delta_Ah)

    self.o.w.assign_add(delta_Wo) # DA AGGIORNARE DOPO INSIEME
    self.h.w.assign_add(delta_Wh)
    
    return {
        "Error": tf.squeeze(self.error.value()),
        "Temperature": tf.squeeze(self.temperature.value()),
        "Hidden weights": tf.squeeze(self.h.w.value()),
        "Output weights": tf.squeeze(self.o.w.value())
        }

In [168]:
# Construct an instance of CustomModel
input = tf.constant([0.01, 0.00, 0.03, 0.22, 0.87, 0.26, 0.42, 0.16])
model = MQSelfReflexiveNetwork(learning_rate=0.1, initial_temperature=1.0, bias_value=0.0001)

In [169]:
model.compile()
model_history = model.fit(x = input, epochs=1000, shuffle=False, verbose = 0)

In [170]:
df_model_history = pd.DataFrame(model_history.history)
df_model_history

,Error,Temperature,Hidden weights,Output weights
0,2.776704,1.0,"[[0.49997163, 0.49978432, 0.4999901, 0.5, 0.49...","[[0.58974934, 0.5897156, 0.5896984, 0.5896786,..."
1,2.776832,1.0,"[[0.49995586, 0.49966452, 0.4999846, 0.5, 0.49...","[[0.58959305, 0.58955276, 0.58953345, 0.589512..."
2,2.776905,1.0,"[[0.4999344, 0.49950147, 0.4999771, 0.5, 0.499...","[[0.58940077, 0.58933747, 0.589306, 0.5892706,..."
3,2.777002,1.0,"[[0.49991548, 0.4993576, 0.4999705, 0.5, 0.499...","[[0.5892217, 0.58914423, 0.58910656, 0.5890649..."
4,2.777088,1.0,"[[0.49989536, 0.49920478, 0.4999635, 0.5, 0.49...","[[0.5890373, 0.58894116, 0.58889395, 0.5888412..."
...,...,...,...,...
995,0.000029,1.0,"[[0.46506232, 0.22363521, 0.488263, 0.5, 0.163...","[[0.013133439, 0.07404508, 0.063823335, 0.0525..."
996,0.000028,1.0,"[[0.4650625, 0.2236348, 0.488263, 0.5, 0.16307...","[[0.013115493, 0.07402704, 0.063812345, 0.0525..."
997,0.000026,1.0,"[[0.46506268, 0.22363438, 0.488263, 0.5, 0.163...","[[0.013097909, 0.07400939, 0.063801594, 0.0525..."
998,0.000025,1.0,"[[0.46506286, 0.22363397, 0.488263, 0.5, 0.163...","[[0.013080682, 0.07399211, 0.063791074, 0.0525..."


In [171]:
# Final error book : 0.00000724

In [172]:
df_model_history.to_csv("model_history.csv")

In [173]:
dvf = abs(model.get_weights()[1][0] - model.get_weights()[1][1])
dvf

array([0.97823143, 0.05024862, 0.29341382, 0.0324029 , 0.20733826,
       0.30383378, 0.07355888, 0.25453845, 0.2440373 , 0.01069883,
       0.2226714 , 0.02170748, 0.47348464, 0.99333036], dtype=float32)

In [174]:
dva = abs(model.get_weights()[0][0,:] - model.get_weights()[0][1,:])
dva

array([0.84000003, 0.03999999, 0.24999999, 0.03      , 0.16      ,
       0.26      , 0.06      , 0.22      , 0.21      , 0.01      ,
       0.19      , 0.02      , 0.39      , 0.86      ], dtype=float32)

In [175]:
dvf

array([0.97823143, 0.05024862, 0.29341382, 0.0324029 , 0.20733826,
       0.30383378, 0.07355888, 0.25453845, 0.2440373 , 0.01069883,
       0.2226714 , 0.02170748, 0.47348464, 0.99333036], dtype=float32)

In [176]:
dvf = [round(x, 3) for x in dvf]

In [177]:
dvf_book = [0.97, 0.05, 0.29, 0.03, 0.17, 0.3, 0.06, 0.24, 0.24, 0.01, 0.21, 0.03, 0.43, 1.00]

In [178]:
# with 3 digits
df_diff = pd.DataFrame({"dva":dva, "dvf":dvf, "dvf_book":dvf_book})
df_diff

,dva,dvf,dvf_book
0,0.84,0.978,0.97
1,0.04,0.050,0.05
2,0.25,0.293,0.29
3,0.03,0.032,0.03
4,0.16,0.207,0.17
5,0.26,0.304,0.30
6,0.06,0.074,0.06
7,0.22,0.255,0.24
8,0.21,0.244,0.24
9,0.01,0.011,0.01


In [179]:
df_diff.to_csv("results.csv")

In [180]:
# standard deviation
df_diff.std()

dva         0.276390
dvf         0.320547
dvf_book    0.320581
dtype: float64

In [181]:
# abs % error std
(abs(df_diff.std().iloc[1] - 0.320581) / 0.320581 ) * 100

0.010759266647686935

In [182]:
dvf = [round(x, 2) for x in dvf]

In [183]:
# with 2 digits
df_diff = pd.DataFrame({"dva":dva, "dvf":dvf, "dvf_book":dvf_book})
df_diff

,dva,dvf,dvf_book
0,0.84,0.98,0.97
1,0.04,0.05,0.05
2,0.25,0.29,0.29
3,0.03,0.03,0.03
4,0.16,0.21,0.17
5,0.26,0.30,0.30
6,0.06,0.07,0.06
7,0.22,0.26,0.24
8,0.21,0.24,0.24
9,0.01,0.01,0.01


In [184]:
fig = px.bar(df_diff,title="dva - dvf - dvf_book", barmode="group")
fig.update_layout( xaxis_title='dimension', yaxis_title='value')
fig.write_html("dva - dvf - dvf_book.html")
fig.write_image("barplot.png")
fig.show()

In [185]:
fig = px.bar(
    (df_diff["dvf"]-df_diff["dvf_book"]) / df_diff["dvf"],
    title="% differences between dvf and dvafbook",
    barmode="group",
    text_auto=".2",
    color = (df_diff["dvf"]-df_diff["dvf_book"])/df_diff["dvf"],
    color_continuous_scale="RdBu_r"
    )
fig.update_layout( xaxis_title='dimension', yaxis_title='% error')
fig.write_html("% differences between dvf and dvafbook.html")
fig.write_image("barplot2.png")
fig.show()

In [186]:
import plotly.figure_factory as ff
hist_data = [((df_diff["dvf"]-df_diff["dvf_book"])/df_diff["dvf"]).to_list()]
group_labels = ['distplot'] # name of the dataset

fig = ff.create_distplot(hist_data, group_labels,bin_size=0.2)
fig.update_layout(title_text="distribution of the % differences")
fig.write_html("Error and temperature.html")
fig.write_image("distplot.png")
fig.show()

In [187]:
fig = px.line(df_model_history[["Error","Temperature"]])
fig.update_layout(title="Error and temperature",
                   xaxis_title='Epoch',
                   yaxis_title='',
                   hovermode='x unified')
fig.write_html("Error and temperature.html")
fig.write_image("error_temperature.png")
fig.show()

---